In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
from __future__ import division
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

In [15]:
global X_train, y_train, X_dev, y_dev, X_test, y_test

In [16]:
def read_data(train_data_path, dev_data_path, test_data_path):
    global X_train, y_train, X_dev, y_dev, X_test, y_test
    train_dataset = pd.read_csv(train_data_path, header = None)
    dev_dataset = pd.read_csv(dev_data_path, header = None)
    test_dataset = pd.read_csv(test_data_path, header = None)
    
    concat_set = pd.concat([train_dataset, dev_dataset, test_dataset], keys=[0,1,2])
    
    temp = pd.get_dummies(concat_set, columns = [1,2,3,4,5,6,8])

    train, dev, test = temp.xs(0), temp.xs(1), temp.xs(2)

    y_train = train[[9]]
    y_dev = dev[[9]]
    y_test = test[[9]]

    y_train = y_train.values.reshape((y_train.shape[0]))
    y_dev = y_dev.values.reshape((y_dev.shape[0]))
    y_test = y_test.values.reshape((y_test.shape[0]))

    train = train.drop([9], axis = 1)
    dev = dev.drop([9], axis = 1)
    test = test.drop([9], axis = 1)
    
    X_train = train.iloc[:, :].values

    X_dev = dev.iloc[:, :].values

    X_test = test.iloc[:, :].values

    # Encode Y
    labelencoder_y = LabelEncoder()
    y_train = labelencoder_y.fit_transform(y_train)
    y_dev = labelencoder_y.fit_transform(y_dev)
    y_test = labelencoder_y.fit_transform(y_test)

    # Convert 0 to -1
    y_train[y_train == 0] = -1
    y_dev[y_dev == 0] = -1
    y_test[y_test == 0] = -1

In [21]:
def dtcv(max_depth):
    global X_train, y_train, X_dev, y_dev, X_test, y_test
    val = cross_val_score(
        DecisionTreeClassifier(max_depth = max_depth),
        X_train, y_train, 'f1', cv = 2
    ).mean()

    return val


In [18]:
def bagcv(n_estimators):
    global X_train, y_train, X_dev, y_dev, X_test, y_test
    val = cross_val_score(
        BaggingClassifier(n_estimators = int(n_estimators)
        ),
        X_train, y_train, 'f1', cv = 2
    ).mean()
    
    return val

In [19]:
def boostcv(n_estimators):
    global X_train, y_train, X_dev, y_dev, X_test, y_test
    val = cross_val_score(
        AdaBoostClassifier(n_estimators=int(n_estimators)
        ),
        X_train, y_train, 'f1', cv=2
    ).mean()
    return val

In [22]:
if __name__ == "__main__":
    
    train_data_path = 'income-data/income.train.txt' 
    dev_data_path = 'income-data/income.dev.txt'
    test_data_path = 'income-data/income.test.txt'
    
    read_data(train_data_path, dev_data_path, test_data_path)
    
    dtBO = BayesianOptimization(dtcv, {'max_depth': (1, 100)})
    
    dtBO.explore({'max_depth': [1, 2, 3, 5, 10, 20, 50, 100]})

    bagBO = BayesianOptimization(
        bagcv,
        {'n_estimators': (10, 250)}
    )

    dtBO.maximize(n_iter=50)
    
    print('-' * 53)
    
    bagBO.maximize(n_iter=50)

    print('-' * 53)
    
    print('Final Results')
    print('DT: %f' % dtBO.res['max']['max_val'])
    print('BAG: %f' % bagBO.res['max']['max_val'])

Initialization
-------------------------------------------
 Step |   Time |      Value |   max_depth | 
    1 | 00m00s |    0.00000 |      1.0000 | 


/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


    2 | 00m00s |    0.33508 |      2.0000 | 
    3 | 00m00s |    0.48967 |      3.0000 | 
    4 | 00m00s |    0.56812 |      5.0000 | 
    5 | 00m00s |    0.59210 |     10.0000 | 
    6 | 00m00s |    0.54851 |     20.0000 | 
    7 | 00m00s |    0.54821 |     50.0000 | 
    8 | 00m00s |    0.54882 |    100.0000 | 
    9 | 00m00s |    0.54791 |     41.9327 | 
   10 | 00m00s |    0.54444 |     21.2317 | 
   11 | 00m00s |    0.56103 |     22.6560 | 
   12 | 00m00s |    0.54775 |     71.0478 | 
   13 | 00m00s |    0.55082 |     91.5552 | 
Bayesian Optimization
-------------------------------------------
 Step |   Time |      Value |   max_depth | 
   14 | 00m01s |    0.54966 |     60.8689 | 
   15 | 00m01s |    0.54945 |     81.2003 | 
   16 | 00m01s |    0.54550 |     32.8869 | 
   17 | 00m01s |    0.54736 |     55.4896 | 
   18 | 00m01s |    0.54485 |     86.3323 | 
   19 | 00m01s |    0.54423 |     66.1321 | 
   20 | 00m01s |    0.55044 |     76.1424 | 
   21 | 00m01s |    0.54565 |     

/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.90946448e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


   30 | 00m02s |    0.58830 |      7.7706 | 
   31 | 00m02s |    0.54647 |     52.6300 | 
   32 | 00m03s |    0.56106 |     16.9355 | 
   33 | 00m03s |    0.54671 |     63.6201 | 
   34 | 00m04s |    0.54600 |     78.8202 | 
   35 | 00m03s |    0.58755 |     12.2129 | 
   36 | 00m03s |    0.54758 |     98.2102 | 
   37 | 00m03s |    0.55001 |     35.2498 | 
   38 | 00m03s |    0.54903 |     83.7479 | 
   39 | 00m04s |    0.54912 |     68.6169 | 
   40 | 00m03s |    0.54801 |     43.9662 | 
   41 | 00m04s |    0.54372 |     30.5532 | 
   42 | 00m04s |    0.54915 |     93.7645 | 
   43 | 00m04s |    0.54729 |     39.7585 | 


...

   54 | 00m28s |    0.60107 |       111.1079 | 


/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00691522]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00035209]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 50, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


   55 | 00m47s |    0.59883 |       247.7509 | 


/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00174807]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00061208]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 45, 'nit': 1, 'warnflag': 2}
  " state: %s" % convergence_dict)
/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:335: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


-----------------------------------------------------
Final Results
DT: 0.592898
BAG: 0.601605
